In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'dailytelegraph_com_au'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_rss_url = 0
publish_source = 'dailytelegraph.com.au'
country = 'Australia'
language = 'English'
images_path = []

rss_pages = c.download_page('https://www.dailytelegraph.com.au/help-rss')
rss_pages = c.scrap('class="header g_wrapper(.*?)</a></p>',rss_pages)
for _ in rss_pages.split('<a href=')[1:]:
    
    rss_url = c.scrap('"(.*?)"', _)
    if 'rss' not in rss_url:
        continue
    
    logger.info(f'Fetching Rss url {rss_url}\n')
    rss_page = c.download_page(rss_url)
    
    if rss_page.startswith('Unable to fetch'):
        logger.info(rss_page) # writes error message with error code
        unable_to_fetch_rss_url += 1
        continue

    for i in rss_page.split('<item>')[1:]:

        # source_link
        source_link = c.scrap("<link>(.*?)<", i)

        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))
        page = c.download_page(source_link)
        if page.startswith('Unable to fetch'):
                logger.info(page) # writes error message with error code
                unable_to_fetch_article_url += 1
                continue
       
        # source_headline
        source_headline = c.scrap('<title>(.*?)</title>', i)
        source_headline = source_headline.replace("&#8217;","'")
        
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            # print(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
         # Date and time
        pub_date, publish_time = '', ''

        try:
            date_time_str = c.scrap('<meta\s*name="article:publicationdate"\s*content="(.*?)\.', page) # 20210330080500
            date_time_str = re.sub('[^\d]','',date_time_str,re.S)                 
            date_time_obj = datetime.strptime(date_time_str, '%Y%m%d%H%M%S')
            # Singapore is 2 hours and 30 minutes ahead of India
            ist_date_time = date_time_obj - timedelta(hours = 2,minutes = 30)  # utc+8 to ist
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass
        
        if not pub_date:
            pub_date = created_on
            publish_time = current_time
 
        
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            # print(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
        if pub_date != created_on:
            break    

        logger.info(f'Fetching {source_link}\n')

        # source_content          
        source_content = c.scrap('class="description g_font-body-m">(.*?)<ul\s*id="share-tools"', page)
        if not source_content:
            source_content = c.scrap('<meta\s*property="og:description"\s*content="(.*?)<meta\s*property="og:url"',page)
        if not source_content:
            source_content = source_headline
        source_content = re.sub('Advertisement' , '', source_content, flags=re.S)
        source_content = c.strip_html(source_content)

        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')
            # print(f'Skipping due to content {source_link}\n')
            skipped_due_to_content += 1
            continue

        # journalist
        journalist =  c.scrap('<meta\s*name="article:author"\s*content="(.*?)"',page)
        if not journalist: journalist = 'NA'

        # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []

        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

        cl_data.insert_one(clientdata)  
        no_of_data += 1
        

logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch rss url: {unable_to_fetch_rss_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')


http://www.dailytelegraph.com.au/news/breaking-news/rss
https://www.dailytelegraph.com.au/lifestyle/health/chief-medical-officer-paul-kelly-tells-states-to-not-stockpile-second-dose-of-pfizer-vaccine/news-story/e7733671c50350ed33026924c9b7c36d
Top doctor’s swipe at Qld over vaccine
https://www.dailytelegraph.com.au/breaking-news/mert-ney-michaela-dunns-killer-tells-court-he-was-obsessed-with-christchurch-shooting-footage/news-story/50f93e52aba9015450f98d858104aa78
CBD stabber’s outburst in court
https://www.dailytelegraph.com.au/sport/afl/collingwood-star-jordan-de-goey-in-court-on-driving-charges/news-story/8b844b6b345c131b5648afb2ea930dd8
Collingwood star on driving charges
https://www.dailytelegraph.com.au/travel/travel-news/wa-premier-mark-mcgowan-warns-nsw-government-against-making-a-covid19-mistake/news-story/3ff7c9a7574c858eb85a58119d6e17de
Premier warning on NSW virus ‘mistake’
https://www.dailytelegraph.com.au/business/companies/officeworks-boss-on-the-covid-shopping-frenzy-su

‘V8 playboy’ James Courtney steps out with new girlfriend
https://www.dailytelegraph.com.au/entertainment/sydney-confidential/two-of-sydneys-oldschool-italian-eateries-rising-up-after-pandemic/news-story/06d9f3656ae43c384f8acac0b23ee9f8
Celebrities still drawn to Sydney’s Italian eateries
https://www.dailytelegraph.com.au/entertainment/sydney-confidential/2gb-radio-host-ray-hadley-gets-married-to-sophie-baird-at-mount-white/news-story/f2b672b6498d4cbe8c12dcf5170ea104
Third time lucky as 2GB’s Ray Hadley ties the knot
https://www.dailytelegraph.com.au/entertainment/sydney-confidential/orasome-fans-left-in-awe-of-ritas-hot-new-look/news-story/55f3262d3b54bdbb4189ea817decd692
Ora-some! Fans left in awe of Rita’s hot new look
http://www.dailytelegraph.com.au/entertainment/celebrity/rss
https://www.dailytelegraph.com.au/entertainment/celebrity/piers-morgan-rants-about-meghan-markle-and-oprah-winfrey-again/news-story/474987dc7db9b97388ae5b72b2390f6b
‘Fake’: Piers lashes Meghan and Oprah
http